In [6]:
import glob
import shutil 
from netCDF4 import Dataset
import numpy as np
import os
import yaml 

from obspy.core import Stream, Trace, UTCDateTime, Stats
from obspy.core.event import read_events


In [7]:
# read event location
with open('../Runs/CartesianStandardRandomModel0005/input/inparam.output.yaml', 'r') as file:
    output_yaml = yaml.load(file, Loader=yaml.FullLoader)

In [8]:
output_yaml['list_of_station_groups'][0]['Synthetic_Stations']['temporal']['time_window']


[0, 2.0]

In [34]:
# data dir
# ModelNameList = ['CartesianModel0007_5pct','CartesianModel0007_10pct','CartesianModel0007_20pct']
ModelNameList = []
for imodel in np.arange(0,20):
    ModeName = 'CartesianStandardRandomModel%04d' %imodel
    ModelNameList.append(ModeName)

# wave dimension to animation
output_channel = 'X'
# wave_dim_s = output_channel.index('s')
# wave_dim_p = output_channel.index('p')
# wave_dim_z = output_channel.index('z')
wave_dim_X = output_channel.index('X')

for ModelName in ModelNameList:
    # data_dir = '../Runs/%s/output/elements/orthogonal_azimuthal_slices' %ModelName
    RunPath = '../Runs/%s' %ModelName

    NETCDFDir = '../DataSet/%s' %ModelName
    if not os.path.exists(NETCDFDir):
        os.mkdir(NETCDFDir)
    StationInfo = np.loadtxt(RunPath+'/input/Synthetic_Stations_Cartesian.txt',dtype=str, skiprows=3)
    stalatlon_dict = {}
    for item in StationInfo:
        stkey = item[1]+'.'+item[0]
        x = float(item[2])
        y = float(item[3])
        if stkey not in stalatlon_dict.keys():
            stalatlon_dict[stkey] = []
        stalatlon_dict[stkey].append((x, y))


    StartTime, EndTime = 0, 2.0    # note in Second
    resample_rate = 0.02    # note in Second
    lowpass_freq = 20    # note in Hz

    PointPerTrace = int((EndTime - StartTime)/resample_rate)+1

    istation = 0

    if os.path.exists(NETCDFDir+'/seismogram.nc'):
        os.remove(NETCDFDir+'/seismogram.nc')

    nc = Dataset(NETCDFDir+'/seismogram.nc', 'w')
    nc.createDimension('npoint', size=PointPerTrace*len(stalatlon_dict))
    nc.createVariable('x', float, dimensions=('npoint'))
    nc.createVariable('y', float, dimensions=('npoint'))
    nc.createVariable('z', float, dimensions=('npoint'))
    nc.createVariable('time', float, dimensions=('npoint'))
    nc.createVariable('X', float, dimensions=('npoint'))


    GSNDir = RunPath + '/output/stations/Synthetic_Stations'

    # read rank-station info
    rank_station_info = np.loadtxt(GSNDir + '/rank_station.info', dtype=str, skiprows=1)

    # dict: mpi-rank -> [station keys]
    rank_station_dict = {}
    for item in rank_station_info:
        rank = item[0]
        stkey = item[1]
        inrank_index = item[2]
        # initialize with an empty array if rank does not exists in rank_station_dict
        if rank not in rank_station_dict.keys():
            rank_station_dict[rank] = []
        # append the station
        rank_station_dict[rank].append([stkey, inrank_index])
        
    # loop over mpi-ranks to read data
    for rank in rank_station_dict.keys():
        f = Dataset(GSNDir + '/axisem3d_synthetics.nc.rank%s' %rank, 'r')
        time = f.variables['data_time'][:]

        for [StationName, inrank_index] in rank_station_dict[rank]:
            stax = stalatlon_dict[StationName][0][0]
            stay = stalatlon_dict[StationName][0][1]
            staz = 0

            # trace header
            stats = Stats()
            stats.starttime = UTCDateTime(time[0])
            stats.delta = UTCDateTime(time[1] - time[0])
            stats.npts = len(time)

            # stream
            stream = Stream()
            for ich, ch in enumerate(output_channel):
                stats.channel = ch  
                # default unit is km
                stream.append(Trace(f.variables['data_wave'][int(inrank_index)][ich], header=stats))


            stream.filter('lowpass', freq=lowpass_freq)
            stream.resample(1/resample_rate)
            # stream = stream.slice(UTCDateTime(int(arrivals[0].time)+StartTime), UTCDateTime(int(arrivals[0].time)+EndTime))
            stream = stream.slice(UTCDateTime(StartTime), UTCDateTime(EndTime))
            npoint_persta = len(stream[0].data)
        
            nc['x'][istation*npoint_persta:(istation+1)*npoint_persta] = stax
            nc['y'][istation*npoint_persta:(istation+1)*npoint_persta] = stay
            nc['z'][istation*npoint_persta:(istation+1)*npoint_persta] = staz

            # nc['time'][istation*npoint_persta:(istation+1)*npoint_persta] = stream[0].times() + int(arrivals[0].time) + StartTime
            nc['time'][istation*npoint_persta:(istation+1)*npoint_persta] = stream[0].times()

            nc['X'][istation*npoint_persta:(istation+1)*npoint_persta] = stream[0].data

            istation += 1

    nc.close()

    print(NETCDFDir+'/seismogram.nc', " set up and saved!")

../DataSet/CartesianStandardRandomModel0000/seismogram.nc  set up and saved!
../DataSet/CartesianStandardRandomModel0001/seismogram.nc  set up and saved!
../DataSet/CartesianStandardRandomModel0002/seismogram.nc  set up and saved!
../DataSet/CartesianStandardRandomModel0003/seismogram.nc  set up and saved!
../DataSet/CartesianStandardRandomModel0004/seismogram.nc  set up and saved!
../DataSet/CartesianStandardRandomModel0005/seismogram.nc  set up and saved!
../DataSet/CartesianStandardRandomModel0006/seismogram.nc  set up and saved!
../DataSet/CartesianStandardRandomModel0007/seismogram.nc  set up and saved!
../DataSet/CartesianStandardRandomModel0008/seismogram.nc  set up and saved!
../DataSet/CartesianStandardRandomModel0009/seismogram.nc  set up and saved!
../DataSet/CartesianStandardRandomModel0010/seismogram.nc  set up and saved!
../DataSet/CartesianStandardRandomModel0011/seismogram.nc  set up and saved!
../DataSet/CartesianStandardRandomModel0012/seismogram.nc  set up and saved!

In [33]:
stream[0].times(), stream[0].data

(array([0.  , 0.02, 0.04, 0.06, 0.08, 0.1 , 0.12, 0.14, 0.16, 0.18, 0.2 ,
        0.22, 0.24, 0.26, 0.28, 0.3 , 0.32, 0.34, 0.36, 0.38, 0.4 , 0.42,
        0.44, 0.46, 0.48, 0.5 , 0.52, 0.54, 0.56, 0.58, 0.6 , 0.62, 0.64,
        0.66, 0.68, 0.7 , 0.72, 0.74, 0.76, 0.78, 0.8 , 0.82, 0.84, 0.86,
        0.88, 0.9 , 0.92, 0.94, 0.96, 0.98, 1.  , 1.02, 1.04, 1.06, 1.08,
        1.1 , 1.12, 1.14, 1.16, 1.18, 1.2 , 1.22, 1.24, 1.26, 1.28, 1.3 ,
        1.32, 1.34, 1.36, 1.38, 1.4 , 1.42, 1.44, 1.46, 1.48, 1.5 , 1.52,
        1.54, 1.56, 1.58, 1.6 , 1.62, 1.64, 1.66, 1.68, 1.7 , 1.72, 1.74,
        1.76, 1.78, 1.8 , 1.82, 1.84, 1.86, 1.88, 1.9 , 1.92, 1.94, 1.96,
        1.98]),
 array([ 5.87006285e-07, -1.12208622e-08, -1.81415808e-09, -2.86519043e-09,
        -9.30568052e-10, -1.36698811e-09, -5.48694146e-10, -7.92876981e-10,
        -3.44144752e-10, -5.02078334e-10, -2.18622762e-10, -3.29772799e-10,
        -1.33591162e-10, -2.16148852e-10, -7.11701811e-11, -1.34708645e-10,
        -2.197

In [28]:
stream = Stream()
stream.append(Trace(f.variables['data_wave'][int(inrank_index)][ich], header=stats))
stream.filter('lowpass', freq=lowpass_freq)
stream.resample(1/0.02)

1 Trace(s) in Stream:
...X | 1970-01-01T00:00:00.013522Z - 1970-01-01T00:00:01.993522Z | 50.0 Hz, 100 samples

In [13]:
stream[0]

...X | 1970-01-01T00:00:00.013522Z - 1970-01-01T00:00:00.013522Z | 20.0 s, 1 samples

In [27]:
npoint_persta, PointPerTrace

(1, 41)

In [21]:
import shutil
# Move netcdf file
# shutil.copy(RunPath+'/input/box_tomography.nc',NETCDFDir)
shutil.copy(RunPath+'/tools/UnifromEarthCore.bm',NETCDFDir)

'/raid1/TomoNet/Runs/Sample/model0011/UnifromEarthCore.bm'